# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/runner/.julia/packages/PyCall/L0fLP/src/numpy.jl:67


Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32)
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp=hgh/lda/si-q4)
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               Hartree()
            

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.119678e+02     1.479039e+00
 * time: 0.7850267887115479
     1     1.042655e+01     1.000569e+00
 * time: 2.1563498973846436
     2    -1.146120e+01     1.106066e+00
 * time: 2.790148973464966
     3    -3.437498e+01     7.156064e-01
 * time: 3.762964963912964
     4    -4.789275e+01     5.197951e-01
 * time: 4.690969944000244
     5    -5.702702e+01     1.786307e-01
 * time: 5.627831935882568
     6    -5.988867e+01     7.041744e-02
 * time: 6.269618988037109
     7    -6.090240e+01     6.050856e-02
 * time: 6.926602840423584
     8    -6.132346e+01     5.113017e-02
 * time: 7.545793771743774
     9    -6.164988e+01     2.970936e-02
 * time: 8.174194812774658
    10    -6.186316e+01     2.255139e-02
 * time: 8.804975986480713
    11    -6.200715e+01     1.774065e-02
 * time: 9.454042911529541
    12    -6.207515e+01     1.823549e-02
 * time: 10.078542947769165
    13    -6.212523e+01     1.533545e-02
 * time: 10.710783958435059
   

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671059
    AtomicLocal         -18.8557630
    AtomicNonlocal      14.8522615
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485359 
    Xc                  -19.3336816

    total               -62.261666457572